In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkSQL-Execution-Plan').getOrCreate()

24/12/10 14:54:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/10 14:54:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
dept_df = spark.read.format('csv')\
                .option('header','true')\
                .option('inferSchema', 'true')\
                .load('data/dept.csv')

In [4]:
dept_df.createOrReplaceTempView('dept_df_view')

In [5]:
emp_df = spark.read.format('csv')\
                .option('header','true')\
                .option('inferSchema', 'true')\
                .load('data/emp.csv')

In [6]:
emp_df.createOrReplaceTempView('emp_df_view')

## 조인

In [7]:
query = '''
select * 
from emp_df_view
join dept_df_view
on emp_df_view.deptno = dept_df_view.deptno
'''

viewEmpDept = spark.sql(query)
viewEmpDept.show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|    20|  RESEARCH|  DALLAS|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|    10|A

## 부서위치가 NEW YORK인 직원 목록

In [11]:
viewEmpDept.createOrReplaceTempView('viewEmpDept')

query = '''
select *
from viewEmpDept
where loc = 'NEW YORK'
'''

spark.sql(query).show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|    10|ACCOUNTING|NEW YORK|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|    10|ACCOUNTING|NEW YORK|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|    10|ACCOUNTING|NEW YORK|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+



In [12]:
# 실행계획
spark.sql(query).explain()

== Physical Plan ==
*(2) BroadcastHashJoin [deptno#45], [deptno#16], Inner, BuildRight, false
:- *(2) Filter isnotnull(deptno#45)
:  +- FileScan csv [empno#38,ename#39,job#40,mgr#41,hiredate#42,sal#43,comm#44,deptno#45] Batched: false, DataFilters: [isnotnull(deptno#45)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab11/src/data/emp.csv], PartitionFilters: [], PushedFilters: [IsNotNull(deptno)], ReadSchema: struct<empno:int,ename:string,job:string,mgr:int,hiredate:string,sal:int,comm:int,deptno:int>
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [id=#257]
   +- *(1) Filter ((isnotnull(loc#18) AND (loc#18 = NEW YORK)) AND isnotnull(deptno#16))
      +- FileScan csv [deptno#16,dname#17,loc#18] Batched: false, DataFilters: [isnotnull(loc#18), (loc#18 = NEW YORK), isnotnull(deptno#16)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab11/src/data/dept.csv], PartitionFilters: [], PushedFilters: [IsNotNull(loc), EqualTo(loc,

## 서브쿼리

In [9]:
query = '''
select *
from emp_df_view
where deptno = (
    select deptno
    from dept_df_view
    where loc='NEW YORK')
'''

spark.sql(query).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|
+-----+------+---------+----+----------+----+----+------+



In [10]:
# 실행계획
spark.sql(query).explain()

== Physical Plan ==
*(1) Filter (isnotnull(deptno#45) AND (deptno#45 = Subquery scalar-subquery#240, [id=#177]))
:  +- Subquery scalar-subquery#240, [id=#177]
:     +- *(1) Project [deptno#16]
:        +- *(1) Filter (isnotnull(loc#18) AND (loc#18 = NEW YORK))
:           +- FileScan csv [deptno#16,loc#18] Batched: false, DataFilters: [isnotnull(loc#18), (loc#18 = NEW YORK)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab11/src/data/dept.csv], PartitionFilters: [], PushedFilters: [IsNotNull(loc), EqualTo(loc,NEW YORK)], ReadSchema: struct<deptno:int,loc:string>
+- FileScan csv [empno#38,ename#39,job#40,mgr#41,hiredate#42,sal#43,comm#44,deptno#45] Batched: false, DataFilters: [isnotnull(deptno#45)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab11/src/data/emp.csv], PartitionFilters: [], PushedFilters: [IsNotNull(deptno)], ReadSchema: struct<empno:int,ename:string,job:string,mgr:int,hiredate:string,sal:int,comm:int,deptno:int>




In [13]:
spark.stop()